In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d naseralqaydeh/named-entity-recognition-ner-corpus

Dataset URL: https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus
License(s): DbCL-1.0
  0% 0.00/4.14M [00:00<?, ?B/s]
100% 4.14M/4.14M [00:00<00:00, 547MB/s]


In [ ]:
from zipfile import ZipFile
with ZipFile('named-entity-recognition-ner-corpus.zip','r') as f:
  f.extractall()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
table=pd.read_csv('ner.csv')

In [ ]:
table.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
table=table[['Sentence','POS']]

In [ ]:
table.head()

,Sentence,POS
0,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'..."
1,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ..."
2,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ..."
3,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ..."
4,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D..."


In [ ]:
import ast

In [ ]:
table['POS']=table['POS'].apply(lambda x:ast.literal_eval(x))

In [ ]:
table.head()

,Sentence,POS
0,Thousands of demonstrators have marched throug...,"[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ..."
1,Families of soldiers killed in the conflict jo...,"[NNS, IN, NNS, VBN, IN, DT, NN, VBD, DT, NNS, ..."
2,They marched from the Houses of Parliament to ...,"[PRP, VBD, IN, DT, NNS, IN, NN, TO, DT, NN, IN..."
3,"Police put the number of marchers at 10,000 wh...","[NNS, VBD, DT, NN, IN, NNS, IN, CD, IN, NNS, V..."
4,The protest comes on the eve of the annual con...,"[DT, NN, VBZ, IN, DT, NN, IN, DT, JJ, NN, IN, ..."


In [ ]:
table['Sentence'][1],table['POS'][1]

('Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
 'NNS IN NNS VBN IN DT NN VBD DT NNS WP VBD NNS IN JJ NNS IN `` NNP NN CD NN `` CC `` VB DT NNS . ``')

In [ ]:
table['POS']=table['POS'].apply(lambda x:' '.join(x))

In [ ]:
max_len=max([len(i.split()) for i in table['Sentence']])
max_len_pos=max([len(i.split()) for i in table['POS']])

In [ ]:
max_len,max_len_pos

(104, 104)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
sent_tok=Tokenizer(filters='',oov_token='<OOV>')
pos_tok=Tokenizer(lower=False,filters='')

In [ ]:
sent_tok.fit_on_texts(table['Sentence'])
pos_tok.fit_on_texts(table['POS'])

In [ ]:
len(sent_tok.word_index),len(pos_tok.word_index)

(31818, 42)

In [ ]:
seq=sent_tok.texts_to_sequences(table['Sentence'])

In [ ]:
pos=pos_tok.texts_to_sequences(table['POS'])

In [ ]:
y=pad_sequences(pos,maxlen=max_len,padding='post')

In [ ]:
X=pad_sequences(seq,maxlen=max_len,padding='post')

In [ ]:
X.shape,y.shape

((47959, 104), (47959, 104))

In [ ]:
from tensorflow.keras import layers,models

In [ ]:
X

array([[ 254,    6,  967, ...,    0,    0,    0],
       [1148,    6,  149, ...,    0,    0,    0],
       [  48, 1795,   22, ...,    0,    0,    0],
       ...,
       [  41,   49, 2716, ...,    0,    0,    0],
       [  48,   27,   44, ...,    0,    0,    0],
       [ 366,   74,   18, ...,    0,    0,    0]], dtype=int32)

In [ ]:
y

array([[ 6,  3,  6, ...,  0,  0,  0],
       [ 6,  3,  6, ...,  0,  0,  0],
       [19,  8,  3, ...,  0,  0,  0],
       ...,
       [12, 26,  8, ...,  0,  0,  0],
       [19, 18, 16, ...,  0,  0,  0],
       [ 5,  6,  8, ...,  0,  0,  0]], dtype=int32)

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("np.expand_dims(y, -1) shape:", np.expand_dims(y, 0).shape)

X shape: (47959, 104)
y shape: (47959, 104)
np.expand_dims(y, -1) shape: (1, 47959, 104)


In [ ]:
model=models.Sequential()
model.add(layers.Embedding(len(sent_tok.word_index)+1,100))
model.add(layers.Bidirectional(layers.LSTM(64,dropout=0.2,return_sequences=True)))
model.add(layers.TimeDistributed(layers.Dense(len(pos_tok.word_index)+1,activation='softmax')))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X,y,epochs=5,validation_split=0.2)

Epoch 1/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 297s 238ms/step - accuracy: 0.9083 - loss: 0.4073 - val_accuracy: 0.9912 - val_loss: 0.0300
Epoch 2/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 275s 229ms/step - accuracy: 0.9934 - loss: 0.0229 - val_accuracy: 0.9933 - val_loss: 0.0214
Epoch 3/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 323s 230ms/step - accuracy: 0.9956 - loss: 0.0141 - val_accuracy: 0.9938 - val_loss: 0.0198
Epoch 4/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 323s 231ms/step - accuracy: 0.9965 - loss: 0.0109 - val_accuracy: 0.9940 - val_loss: 0.0198
Epoch 5/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 319s 229ms/step - accuracy: 0.9971 - loss: 0.0091 - val_accuracy: 0.9940 - val_loss: 0.0202


In [ ]:
model.save('pos_model.keras')

In [ ]:
def pos(sentence):
  sent=sent_tok.texts_to_sequences([sentence])
  sent_pad=pad_sequences(sent,maxlen=max_len,padding='post')
  pred=model.predict(sent_pad)
  prediction=np.argmax(pred,-1)
  n=0
  for i in prediction[0]:
    if i!=0:
     n=n+1
    else:
     break
  prediction=prediction[:n]
  pred_sent=pos_tok.sequences_to_texts(prediction)
  print(sentence)
  return pred_sent[0]

In [ ]:
pos(table['Sentence'][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .


'NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC VB DT NN IN JJ NNS IN DT NN .'

In [ ]:
pos("I can't believe it's already September .")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
I can't believe it's already September .


'PRP NNS VBP NNP RB NNP .'

In [ ]:
pos("She is reading a book in the library .")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
She is reading a book in the library .


'PRP VBZ VBG DT NN IN DT NN .'

In [ ]:
pos("The cat sat on the mat .")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The cat sat on the mat .


'DT NN VBD IN DT NN .'

In [ ]:
pos("He quickly ran to catch the bus")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
He quickly ran to catch the bus


'PRP RB VBD TO VB DT NN'

In [ ]:
pos("If you had told me earlier, I would n't have gone.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
If you had told me earlier, I would n't have gone.


'IN PRP VBD VBN PRP NNS PRP MD RB VB JJ'

In [ ]:
pos('Despite being tired , she completed the assignment on time')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Despite being tired , she completed the assignment on time


'IN VBG JJ , PRP VBD DT NN IN NN'

In [ ]:
import pickle
pickle.dump(sent_tok,open('sent_token.pkl','wb'))
pickle.dump(pos_tok,open('pos_token.pkl','wb'))